In [45]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

# Replace 'your_file.csv' with the path to your CSV file
data = pd.read_csv('test.csv')
data.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,32.6909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,3.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,27.0552,NaN,NaN,Fall,2.340,Fall,46.0,64.0,Summer,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,NaN,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,45.9966,NaN,NaN,Winter,2.451,Summer,31.0,45.0,Winter,0.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,Summer,1.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
threshold = 0.5
data_filtered = data.loc[:, data.isnull().mean() < threshold]
data_filtered.columns.tolist()

['id',
 'Basic_Demos-Enroll_Season',
 'Basic_Demos-Age',
 'Basic_Demos-Sex',
 'Physical-Season',
 'Physical-BMI',
 'Physical-Height',
 'Physical-Weight',
 'Physical-Diastolic_BP',
 'Physical-HeartRate',
 'Physical-Systolic_BP',
 'FGC-Season',
 'FGC-FGC_CU',
 'FGC-FGC_CU_Zone',
 'FGC-FGC_PU',
 'FGC-FGC_PU_Zone',
 'FGC-FGC_SRL',
 'FGC-FGC_SRL_Zone',
 'FGC-FGC_SRR',
 'FGC-FGC_SRR_Zone',
 'FGC-FGC_TL',
 'FGC-FGC_TL_Zone',
 'PreInt_EduHx-Season',
 'PreInt_EduHx-computerinternet_hoursday']

In [36]:
len(data_filtered.columns)

24

In [30]:
data_withNaN = data_filtered.loc[:, data.isna().any()]
data_withNaN.columns.tolist()

['Physical-Season',
 'Physical-BMI',
 'Physical-Height',
 'Physical-Weight',
 'Physical-Diastolic_BP',
 'Physical-HeartRate',
 'Physical-Systolic_BP',
 'FGC-Season',
 'FGC-FGC_CU',
 'FGC-FGC_CU_Zone',
 'FGC-FGC_PU',
 'FGC-FGC_PU_Zone',
 'FGC-FGC_SRL',
 'FGC-FGC_SRL_Zone',
 'FGC-FGC_SRR',
 'FGC-FGC_SRR_Zone',
 'FGC-FGC_TL',
 'FGC-FGC_TL_Zone',
 'PreInt_EduHx-Season',
 'PreInt_EduHx-computerinternet_hoursday']

In [35]:
data_withNaN.columns.shape[0]

20

In [42]:
columns_to_select = [
    'id', 'Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
    'Physical-BMI', 'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
    'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_PU', 'FGC-FGC_PU_Zone',
    'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone',
    'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'PreInt_EduHx-computerinternet_hoursday'
]
data_tobeused = data_filtered[columns_to_select]
data_tobeused.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,Physical-BMI,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,PreInt_EduHx-computerinternet_hoursday
0,00008ff9,Fall,5,0,16.877316,NaN,NaN,NaN,0.0,0.0,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,3.0
1,000fd460,Summer,9,0,14.035590,75.0,70.0,122.0,3.0,0.0,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,0.0
2,00105258,Summer,10,1,16.648696,65.0,94.0,117.0,20.0,1.0,7.0,1.0,10.0,1.0,10.0,1.0,5.0,0.0,2.0
3,00115b9f,Winter,9,0,18.292347,60.0,97.0,117.0,18.0,1.0,5.0,0.0,7.0,0.0,7.0,0.0,7.0,1.0,0.0
4,0016bb22,Spring,18,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
data_tobeused.columns.tolist()

['id',
 'Basic_Demos-Enroll_Season',
 'Basic_Demos-Age',
 'Basic_Demos-Sex',
 'Physical-BMI',
 'Physical-Diastolic_BP',
 'Physical-HeartRate',
 'Physical-Systolic_BP',
 'FGC-FGC_CU',
 'FGC-FGC_CU_Zone',
 'FGC-FGC_PU',
 'FGC-FGC_PU_Zone',
 'FGC-FGC_SRL',
 'FGC-FGC_SRL_Zone',
 'FGC-FGC_SRR',
 'FGC-FGC_SRR_Zone',
 'FGC-FGC_TL',
 'FGC-FGC_TL_Zone',
 'PreInt_EduHx-computerinternet_hoursday']

In [118]:
X = data_tobeused[['Basic_Demos-Age',
 'Basic_Demos-Sex']]
y = data_tobeused['Physical-BMI']
X_train = X.loc[y.dropna().index,:]
y_train = y.dropna()
X_test = X[y.isna()]
y_test = y[y.isna()]
# Ensure all data in X_train is numeric
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1) 

X_test = X_test.apply(pd.to_numeric, errors='coerce')
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1) 
model = nn.Linear(2, 1)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training
for epoch in range(len(X_train)):
    predictions = model(X_train)
    loss = criterion(predictions, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


predictions = model(X_test)
print(predictions)
print(X_test.shape[0])

tensor([[38.5691],
        [32.2333],
        [40.6362],
        [24.0993],
        [23.9649],
        [26.0320],
        [21.8978]], grad_fn=<AddmmBackward0>)
7


In [90]:
X_train = X.loc[y.dropna().index,:]
X_train = X.loc[y.dropna().index,:]
print(X_train.values)
y = data_tobeused['Physical-BMI']
y_train = y.dropna()
y_train.values

[[ 5  0]
 [ 9  0]
 [10  1]
 [ 9  0]
 [13  1]
 [10  0]
 [10  1]
 [13  0]
 [12  0]
 [ 7  0]
 [ 5  1]
 [10  1]
 [ 6  0]]


array([16.87731569, 14.03559028, 16.64869606, 18.29234694, 22.27995198,
       19.66076033, 16.86128647, 21.07906523, 15.54411111, 29.31577503,
       17.28450413, 19.89315702, 30.09464889])

In [54]:
X = data_tobeused[['Basic_Demos-Age', 'Basic_Demos-Sex']].values
X

array([[ 5,  0],
       [ 9,  0],
       [10,  1],
       [ 9,  0],
       [18,  1],
       [13,  1],
       [10,  0],
       [10,  1],
       [15,  0],
       [19,  1],
       [11,  1],
       [11,  0],
       [13,  0],
       [12,  0],
       [12,  0],
       [ 7,  0],
       [ 5,  1],
       [10,  1],
       [ 6,  0],
       [10,  0]])